#**RAG Basic Example with Paragraph**

Think of RAG as:

“Looking up a relevant paragraph (retrieval), then using it to write an answer (generation) — like open-book exam answers!”

In [4]:
!pip install gradio openai scikit-learn

#**Import Statement**

In [3]:
import gradio as gr
from openai import OpenAI
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

###**Retrive API key from Secrets and Set as an ENV**

In [5]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [6]:
# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [9]:
client=OpenAI()

###**Define Paragraph**

In [8]:
paragraph = """Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can perform tasks such as recognizing speech, understanding natural language, making decisions, and identifying patterns. These systems are powered by algorithms and vast amounts of data, enabling them to improve their performance over time. AI has a wide range of applications, including autonomous vehicles, medical diagnostics, customer service, and more. As AI continues to evolve, it holds the potential to revolutionize industries and change the way humans interact with technology.
            Machine Learning (ML) is a subset of Artificial Intelligence that focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data. Unlike traditional programming, where a developer explicitly tells the computer how to perform tasks, ML algorithms improve their performance by analyzing patterns and making adjustments without human intervention. ML is widely used in applications such as recommendation systems, fraud detection, image recognition, and natural language processing. The field continues to grow rapidly, driven by advancements in computational power, data availability, and algorithmic improvements."""

###**Embedding Function**

In [10]:
def get_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return np.array(response.data[0].embedding)


In [15]:
# To view the embeddings (Optional)
embed=get_embeddings("Hello")
print(embed)

[-0.0218327  -0.00703757 -0.02864325 ... -0.02047059 -0.00427119
 -0.00318799]


###**Paragraph Retrieval**

In [11]:
def retrieve_relevant_paragraph(paragraph, question):
    paragraph_embedding = get_embeddings(paragraph)
    question_embedding = get_embeddings(question)
    similarity = cosine_similarity([question_embedding], [paragraph_embedding])[0][0]
    if similarity > 0.7:
        return paragraph
    else:
        return "The question does not match the paragraph well enough."


####**Optional: To verify the retrieval function**

In [18]:
retrieval=retrieve_relevant_paragraph(paragraph,"what is Cruize?")

In [19]:
print(retrieval)

The question does not match the paragraph well enough.


In [21]:
retrieval_para=retrieve_relevant_paragraph(paragraph,"what is AI?")

In [22]:
print(retrieval_para)

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can perform tasks such as recognizing speech, understanding natural language, making decisions, and identifying patterns. These systems are powered by algorithms and vast amounts of data, enabling them to improve their performance over time. AI has a wide range of applications, including autonomous vehicles, medical diagnostics, customer service, and more. As AI continues to evolve, it holds the potential to revolutionize industries and change the way humans interact with technology. 
            Machine Learning (ML) is a subset of Artificial Intelligence that focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data. Unlike traditional programming, where a developer explicitly tells the computer how to perform tasks, ML algorithms improve their performance by analyzing p

###**Answer Generation**

In [12]:
def generate_response(question):
    relevant_paragraph = retrieve_relevant_paragraph(paragraph, question)
    print(relevant_paragraph)
    if relevant_paragraph == "The question does not match the paragraph well enough.":
        return relevant_paragraph
    else:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. Answer questions based on the provided paragraph in 30 words."},
                    {"role": "user", "content": f"Here is the paragraph: {relevant_paragraph}\n\nQuestion: {question}"}
                ]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error: {e}"


###**Launch Gradio App**

In [13]:
with gr.Blocks() as app:
    gr.Markdown("# Chat with a Paragraph 📄🤖")
    gr.Markdown(f"### Paragraph: \n{paragraph}")
    question = gr.Textbox(label="Ask a Question", placeholder="Ask a question about the paragraph...")
    submit_button = gr.Button("Get Answer")
    answer = gr.Textbox(label="Answer", interactive=False)
    submit_button.click(generate_response, inputs=[question], outputs=answer)

app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e82db7b5669a2a2afa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
